In [ ]:
import os
import sys
import subprocess

# --- CONFIGURAZIONE DEL PROGETTO ---
# Modifica queste variabili per clonare il tuo repository specifico
GIT_REPO_URL = "https://github.com/Biobay/DeepLearning/"  # <-- INSERISCI QUI L'URL DEL TUO REPOSITORY
BRANCH_NAME = "RIS-1-CORRETTO"  # <-- INSERISCI QUI IL NOME DEL BRANCH
PROJECT_DIR = "DeepLearning" # <-- NOME DELLA CARTELLA DOPO LA CLONAZIONE

# --- SCRIPT DI SETUP E TRAINING ---

# 1. Clona il repository e si sposta nel branch specificato
if not os.path.exists(PROJECT_DIR):
    print(f"Clonazione del branch '{BRANCH_NAME}' da '{GIT_REPO_URL}'...")
    subprocess.run(['git', 'clone', '--branch', BRANCH_NAME, GIT_REPO_URL, PROJECT_DIR], check=True)
else:
    print(f"La cartella '{PROJECT_DIR}' esiste già. Salto la clonazione.")

# 2. Naviga nella cartella del progetto
os.chdir(PROJECT_DIR)
print(f"Cartella di lavoro corrente: {os.getcwd()}")

# 3. Installa le dipendenze
print("Installazione delle dipendenze da requirements.txt...")
subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], check=True)

# 4. Aggiunge il percorso al path di Python per gli import
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
print("Path di sistema aggiornato.")

# 5. Avvia l'addestramento
print("--- Avvio del processo di addestramento ---")
try:
    from scripts.train import train
    import src.config as config
    print("Moduli del progetto importati con successo!")
    train(config)
    print("--- Processo di addestramento terminato ---")
except ImportError as e:
    print(f"ERRORE: Impossibile importare i moduli. {e}")
except Exception as e:
    print(f"Si è verificato un errore durante l'addestramento: {e}")
